In [1]:
# Clase que genera un punto cuantico triangular o hexagonal de tamano n.

import numpy as np
from math import sqrt
import matplotlib.pyplot as plt

class punto_cuantico():
    
    def __init__(self, n, d=0.0, ang=0.0, addH=False,
                acc = 1.44, CH=1.41):
        # La clase retorna las siguientes caracteristicas:
        self.n = n # Tamano punto cuantico.
        self.acc = acc # Distancia entre carbonos: C-C
        self.CH = CH # Distancia entre C-H.
        self.d = d # Valor en eje Z
        self.ang = ang # Angulo de rotacion
        self.addH = addH # Booleano que agrega coordenadas de Hidrogenos
        self.atomos = None # Vector de atomos: 1='C', 0='H'
        self.coords = None # Matriz de Coordenadas [x,y,z]
        self.coordsC = None # Matriz de Coordenadas de Carbonos
        self.coordsH = None # Matriz de Coordenadas de Hidrogenos
        self.N = None # Cantidad de Carbonos + Hidrogenos
        self.nC = None # Cantidad de Carbonos
        self.nH = None # Cantidad de Hidrogenos
        self.x, self.y, self.z = None, None, None # Coords x,y,z, C+H.
        self.Cx, self.Cy, self.Cz = None, None, None # Coords x,y,z, C.
        self.Hx, self.Hy, self.Hz = None, None, None # Coords x,y,z, H.
        
    # Funcion que rota coordenadas x y y alrededor del punto(0,0)
    def rotacion(self, x,y):
        A = [[np.cos(self.ang), -np.sin(self.ang)],
             [np.sin(self.ang), np.cos(self.ang)]]
        r = [x,y]
        x, y = np.dot(A,r)
        return list(x), list(y)
    
    # Guarda los parametros de los Hidrogenos
    def ParametrosH(self):
        if self.ang!=0.:
            self.Hx, self.Hy = self.rotacion(self.Hx,self.Hy)
        self.atomos += list(np.zeros(len(self.Hx)))
        self.x += self.Hx
        self.y += self.Hy
        self.z += self.Hz
        self.coords = [self.x, self.y, self.z]
        self.coordsH = [self.Hx,self.Hy,self.Hz]
        self.nH = len(self.Hx)
        self.Hx, self.Hy
        self.N = self.nC+self.nH
        return self
    
    # Guarda los parametros de los Carbonos
    def ParametrosC(self, Hex=False):
        self.atomos = list(np.ones(len(self.x)))
        self.coordsC = [self.x,self.y,self.z]
        self.nC = len(self.x)
        if self.ang != 0.:
            self.x, self.y = self.rotacion(self.x,self.y)
        self.Cx, self.Cy, self.Cz = self.x.copy(),self.y.copy(),self.z.copy()
        if self.addH == True:
            if Hex==False:
                return self.addHidrogenosTri()
            else:
                return self.addHidrogenosHex()
        self.N = self.nC
        self.coords = [self.x,self.y,self.z]
        return self
    
    # Agrega hidrogenos a las coordenadas Triangulares.
    def addHidrogenosTri(self):
        t=(0.75*self.n-0.25)*self.acc
        self.Hx, self.Hy  = [0], [t+self.acc/2+self.CH] #Inicializando coords hidrogenos.
        for k in range(self.n):
            self.Hx += [-sqrt(3)/2*((k+1)*self.acc+self.CH),
                        sqrt(3)/2*((k+1)*self.acc+self.CH)]
            self.Hy += [t+k*-1.5*self.acc+self.CH/2, 
                   t+k*-1.5*self.acc+self.CH/2]
        self.Hx += (list(np.linspace(-(self.n-1)*sqrt(3)*self.acc/2,
                                     (self.n-1)*sqrt(3)*self.acc/2, self.n))
               +[self.Hx[self.n*2]] + [self.Hx[self.n*2-1]])
        self.Hy += (list(t+(self.n-1)*-1.5*self.acc*np.ones(self.n)-1.5*self.acc-self.CH) 
                    +[self.Hy[self.n*2]-self.acc-self.CH] 
                    +[self.Hy[self.n*2]-self.acc-self.CH])
        self.Hz = list(np.ones(len(self.Hx))*self.d)
        return self.ParametrosH()
    
    # Funcion que genera las coordenadas de un punto cuantico triangular
    # de tamano n.
    def Triangular(self):
        t = (0.75*self.n-0.25)*self.acc  #Traslacion en "y" para centrar.
        self.x, self.y  = [0], [t+self.acc/2] #Inicializando coordenadas triangulo.
        for k in range(self.n):
            i = k+1
            j = t + k*-1.5*self.acc
            X = np.linspace(-i*sqrt(3)*self.acc/2, i*sqrt(3)*self.acc/2, i+1)
            Y = np.ones(i+1)*j
            self.x += list(X)*2
            self.y += list(Y)+list(Y-self.acc)
        self.x += list(X+sqrt(3)/2*self.acc)
        self.y += list(Y-1.5*self.acc)
        self.z = list(np.ones(len(self.x))*self.d)
        self.x.pop(-1)
        self.y.pop(-1)
        self.z.pop(-1)
        return self.ParametrosC()
    
    # Agrega hidrogenos a las coordenadas Hexagonales.
    def addHidrogenosHex(self):
        t = (1.5*self.n-1.)*self.acc
        self.Hx, self.Hy = [], []
        I = list(np.arange(self.n,self.n*2))
        for k in range(self.n):
            i=I[k]
            j = t + k*-1.5*self.acc
            self.Hx += [-i*sqrt(3)*self.acc/2-sqrt(3)/2*self.CH,
                    i*sqrt(3)*self.acc/2+sqrt(3)/2*self.CH]
            self.Hy += [j+self.CH/2, j+self.CH/2]
        self.Hx += list(np.arange(-(self.n-1)*sqrt(3)*self.acc/2,
                              (self.n-1)*sqrt(3)*self.acc/2+1, 
                               sqrt(3)*self.acc)) 
        self.Hy += list(np.ones(self.n)*t+self.acc/2+self.CH)
        self.Hx += self.Hx
        self.Hy += list(np.array(self.Hy)*-1)
        self.Hz = list(np.zeros(len(self.Hx))*self.d)
        return self.ParametrosH()
    
    # Funcion que genera las coordenadas de un punto cuantico hexagonal
    # de tamano n.
    def Hexagonal(self):
        t = (1.5*self.n-1)*self.acc # Traslacion en "y" para centrar.
        self.x, self.y = [], []
        I = list(np.arange(self.n,self.n*2-1))+list(np.arange(self.n*2-1,self.n-1,-1))
        for k in range(self.n*2-1):
            i=I[k]
            j = t + k*-1.5*self.acc    
            X = np.arange(-i*sqrt(3)*self.acc/2, i*sqrt(3)*self.acc/2+1, sqrt(3)*self.acc)
            Y= np.ones(i+1)*j
            self.x += list(X)+list(X)
            self.y += list(Y)+list(Y-self.acc)
        xx = list(np.arange(-(self.n-1)*sqrt(3)*self.acc/2, 
                             (self.n-1)*sqrt(3)*self.acc/2+1, sqrt(3)*self.acc))
        self.x += xx*2
        self.y += ( list(np.ones(self.n)*max(self.y)+0.5*self.acc)
                   +list(np.ones(self.n)*min(self.y)-0.5*self.acc))
        self.z = list(np.ones(len(self.x))*self.d)
        return self.ParametrosC(True)
    
    def graph(self):
        plt.figure(figsize=(3.9,3.6), dpi=100)
        plt.title("n="+str(self.n))
        plt.plot(self.Cx,self.Cy,'r.')
        if self.addH==True:
            plt.plot(self.Hx,self.Hy,'y.', ms=8)
        plt.show()